## imports


In [ ]:
# !ls /datasets/aurora

In [9]:
! sudo apt remove -y python3-numpy python3-scipy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'python3-scipy' is not installed, so not removed
Package 'python3-numpy' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  cudnn-license fonts-lyx libabsl20210324 libaec0 libboost-dev
  libboost1.74-dev libc-ares2 libdouble-conversion3 libgif7 libhdf5-103-1
  libhdf5-hl-100 libimagequant0 libjs-jquery-ui liblbfgsb0 liblzf1 libmagma2
  libopenblas-dev libopenblas-pthread-dev libopenblas0 libqhull-r8.0 libraqm0
  libsz2 libwebpdemux2 libxsimd-dev python-matplotlib-data python3-absl
  python3-appdirs python3-astunparse python3-beniget python3-bleach
  python3-brotli python3-bs4 python3-cycler python3-einops python3-flatbuffers
  python3-fs python3-fsspec python3-gast python3-grpcio python3-html5lib
  python3-joblib python3-kiwisolver python3-lxml python3-lz4 python3-mpmath
  python3-namex python3-networkx python3-olefile 

In [12]:
!pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu124
!pip install cdsapi dask xarray netcdf4 matplotlib microsoft-aurora boto3 awscli
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
!pip install scikit-learn
# !pip install --upgrade numpy scipy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu124
Defaulting to user installation because normal site-packages is not writeable
  Using cached cdsapi-0.7.5-py2.py3-none-any.whl (12 kB)
  Using cached dask-2024.12.1-py3-none-any.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 62.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 163.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 KB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 KB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 271.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 KB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 24.4 MB/s eta 0:00:00
     ━━━━

tthe next two boxes are used for using aws cli 

In [5]:
!aws --version

aws-cli/1.36.22 Python/3.10.12 Linux/6.8.0-1013-nvidia-64k botocore/1.35.81


In [ ]:
# see awsid

In [1]:
import xarray as xr
from tqdm import tqdm
from aurora import Batch, Metadata
import os
import glob
import re
import boto3
from io import BytesIO
import io
import pandas as pd
import torch.nn.functional as F
from aurora import Aurora
from aurora.normalisation import locations, scales
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim
import torch
import sklearn
import numpy as np
import gc
# from netCDF4 import Dataset
# import h5py

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
torch.cuda.is_available()

True

## importing data

In [ ]:
# see awsid for missing line

# bucket_name = 'globfire-gooddata'
# compression_settings = {'zlib': True, 'complevel': 3}

# surf_comb = xr.open_mfdataset(surf_files, combine='nested')
# encoding_surf = {var: compression_settings for var in surf_comb.data_vars}
# surf_comb.to_netcdf("surf_comb.nc", encoding=encoding_surf)
# s3.upload_file("surf_comb.nc", bucket_name, "surf_comb.nc")

code for joinging 3 days worth of data together to feed into aurora

In [ ]:
download_path = "datasets/aurora"
surf_path = f"../{download_path}/fle"
atmos_path = f"../{download_path}/atmospheric"

static_vars_ds = xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4")
atmos_files = glob.glob(f"../{download_path}/atmospheric/*/*.nc") 
atmos_files_2021 = [f for f in atmos_files if int(f.split('/')[-2]) <= 202112]
surf_files = glob.glob(f"../{download_path}/fle/*.nc") 

In [ ]:
starty = 2015     ## start of 2015 to end of 2021
endy = 2021           
date_range = pd.date_range(start=f'{starty}-01-01', end=f'{endy}-12-31', freq='D')

download_path = "datasets/aurora"
surf_path = f"../{download_path}/fle"
atmos_path = f"../{download_path}/atmospheric"

atmos_files = glob.glob(f"../{download_path}/atmospheric/*/*.nc") 
# atmos_files = glob.glob(f"../{download_path}/atmospheric/(201[0-9][0-9]{2}|202(0|1)[0-9]{2})/*.nc")  
atmos_files_2021 = [f for f in atmos_files if int(f.split('/')[-2]) <= 202112]
surf_files = glob.glob(f"../{download_path}/fle/*.nc") 


days = 3

# ./datasets/aurora/atmospheric/201501/atmospheric_20150111.nc'
#../datasets/aurora/fle/surf_2021-11-17.nc'

In [ ]:
i=0
date_range[i:i + days]

## basic one day batch

In [7]:
variables_to_keep = ['u10', 'v10', 't2m', 'msl', 'lst', 'fire']
date_range = pd.date_range(start=f'{2015}-01-01', end=f'{2015}-1-03', freq='D')
days = 1
for i in tqdm(range(len(date_range) - days + 1), desc = "data loading"):
    date_window = date_range[i:i + days]
    
    atmos_regex = []
    surf_regex = []
    for day in range(days):
        atmos_regex.append( re.compile( rf".*/{date_window[day].strftime('%Y%m')}/atmospheric_{date_window[day].strftime('%Y%m%d')}\.nc$"))
        surf_regex.append( re.compile( rf".*/surf_{date_window[day].strftime('%Y-%m-%d')}\.nc$"))
    
    atmos_filt = [f for f in atmos_files_2021 if any(d.match(f) for d in atmos_regex)]
    surf_filt = [f for f in surf_files if any(d.match(f) for d in surf_regex)]

    print(atmos_filt)
    print(surf_filt)
    
    
    atmos_list = []
    surf_list = []
    
    for day in range(len(atmos_filt)):
        atmos_cur = xr.open_dataset(atmos_filt[day])
        surf_cur = xr.open_dataset(surf_filt[day])
        
        # code for dropping variables. makes code run slower. interpolation so fast that it doest matter. 
        surf_cur = surf_cur.drop_vars([var for var in surf_cur.data_vars if var not in variables_to_keep])

        atmos_list.append(atmos_cur)
        surf_list.append(surf_cur)
    
    atmos_comb = xr.concat(atmos_list, dim="valid_time")
    surf_comb = xr.concat(surf_list, dim="valid_time")
    
static_vars_ds = xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4")

data loading:   0%|          | 0/3 [00:00<?, ?it/s]


NameError: name 'atmos_files_2021' is not defined

In [ ]:

i = 1  # Select this time index in the downloaded data.

batch = Batch(
    surf_vars={
        # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
        # batch dimension of size one.
        "2t": torch.from_numpy(surf_comb["t2m"].values[[i - 1, i]][None]),
        "10u": torch.from_numpy(surf_comb["u10"].values[[i - 1, i]][None]),
        "10v": torch.from_numpy(surf_comb["v10"].values[[i - 1, i]][None]),
        "msl": torch.from_numpy(surf_comb["msl"].values[[i - 1, i]][None]),
        "fire": torch.from_numpy(surf_comb["fire"].values[[i - 1, i]][None]),
        "lst": torch.from_numpy(surf_comb["lst"].values[[i - 1, i]][None]),
    },
    static_vars={
        # The static variables are constant, so we just get them for the first time.
        "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
    },
    atmos_vars={
        "t": torch.from_numpy(atmos_comb["t"].values[[i - 1, i]][None]),
        "u": torch.from_numpy(atmos_comb["u"].values[[i - 1, i]][None]),
        "v": torch.from_numpy(atmos_comb["v"].values[[i - 1, i]][None]),
        "q": torch.from_numpy(atmos_comb["q"].values[[i - 1, i]][None]),
        "z": torch.from_numpy(atmos_comb["z"].values[[i - 1, i]][None]),
    },
    metadata=Metadata(
        lat=torch.from_numpy(surf_comb.latitude.values),
        lon=torch.from_numpy(surf_comb.longitude.values),
        # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        # one value for every batch element.
        time=(surf_comb.valid_time.values.astype("datetime64[s]").tolist()[i],),
        atmos_levels=tuple(int(level) for level in atmos_comb.pressure_level.values),
    ),
)

In [ ]:
# static_vars_ds = xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4")
# surf_vars_ds = xr.open_dataset(f"../{download_path}/fle/surf_2015-01-04.nc", engine="netcdf4")
# atmos_vars_ds = xr.open_dataset(f"../{download_path}/atmospheric/201501/atmospheric_20150104.nc", engine="netcdf4")


j = i+1  # Select this time index in the downloaded data.

surf_size = (720, 1440)
atmos_size = (720, 1440)

surf_vars={
    # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
    # batch dimension of size one.
    "2t": torch.from_numpy(surf_comb["t2m"].values[[j]][None]),
    "10u": torch.from_numpy(surf_comb["u10"].values[[j]][None]),
    "10v": torch.from_numpy(surf_comb["v10"].values[[j]][None]),
    "msl": torch.from_numpy(surf_comb["msl"].values[[j]][None]),
    "fire": torch.from_numpy(surf_comb["fire"].values[[j]][None]),
    "lst": torch.from_numpy(surf_comb["lst"].values[[j]][None]),
}
static_vars={
    # The static variables are constant, so we just get them for the first time.
    "z": torch.from_numpy(static_vars_ds["z"].values[0]),
    "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
    "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
}
atmos_vars={
    "t": torch.from_numpy(atmos_comb["t"].values[[j]][None]),
    "u": torch.from_numpy(atmos_comb["u"].values[[j]][None]),
    "v": torch.from_numpy(atmos_comb["v"].values[[j]][None]),
    "q": torch.from_numpy(atmos_comb["q"].values[[j]][None]),
    "z": torch.from_numpy(atmos_comb["z"].values[[j]][None]),
}

output_size = (720, 1440)

interpolated_surf_vars = {
    key: F.interpolate(value, size=surf_size, mode='bilinear', align_corners=False)
    for key, value in surf_vars.items()
}

interpolated_atmos_vars = {
    key: F.interpolate(value.view(-1, 1, 721, 1440), size=atmos_size, mode='bilinear', align_corners=False).view(value.shape[0], value.shape[1], value.shape[2], 720, 1440)
    for key, value in atmos_vars.items()
}

interpolated_static_vars = {
    key: F.interpolate(value.unsqueeze(0).unsqueeze(0), size=output_size, mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
    for key, value in static_vars.items()
}


batch2 = Batch(
    surf_vars = interpolated_surf_vars,
    static_vars=interpolated_static_vars,
    atmos_vars=interpolated_atmos_vars,
    metadata=Metadata(
        lat=torch.from_numpy(surf_comb.latitude.values),
        lon=torch.from_numpy(surf_comb.longitude.values),
        # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        # one value for every batch element.
        time=(surf_comb.valid_time.values.astype("datetime64[s]").tolist()[i],),
        atmos_levels=tuple(int(level) for level in atmos_comb.pressure_level.values),
    ),
)

## convert to dataset and dataloader 

In [6]:

# using surf_comb is not the best solution
# lstdata = torch.from_numpy(surf_comb["lst"].values[[i - 1, i]][None])
# Normalisation means:
locations["fire"] = 0.0
locations["lst"] = 11484.10859

# Normalisation standard deviations:
scales["fire"] = 1.0
scales["lst"] = 8109.33224

In [7]:
def data_retreive ( atmos_files, surf_files, date_range, days, idx , variables_to_keep = None ):
    date_window = date_range[idx:idx + days]

    atmos_regex = []
    surf_regex = []
    for day in range(days):
        atmos_regex.append( re.compile( rf".*/{date_window[day].strftime('%Y%m')}/atmospheric_{date_window[day].strftime('%Y%m%d')}\.nc$"))
        surf_regex.append( re.compile( rf".*/surf_{date_window[day].strftime('%Y-%m-%d')}\.nc$"))

    atmos_filt = [f for f in atmos_files if any(d.match(f) for d in atmos_regex)]
    surf_filt = [f for f in surf_files if any(d.match(f) for d in surf_regex)]
    atmos_list = []
    surf_list = []

    for day in range(len(atmos_filt)):
        with xr.open_dataset(atmos_filt[day])as atmos_cur, xr.open_dataset(surf_filt[day]) as surf_cur :

            # code for dropping variables. makes code run slower. interpolation so fast that it doest matter. 
            if variables_to_keep is not None:
                surf_cur = surf_cur.drop_vars([var for var in surf_cur.data_vars if var not in variables_to_keep])

            atmos_list.append(atmos_cur.load())
            surf_list.append(surf_cur.load())

    atmos_comb = xr.concat(atmos_list, dim="valid_time")
    surf_comb = xr.concat(surf_list, dim="valid_time")
    return atmos_comb, surf_comb


In [8]:
def feature_label_batch(atmos_comb, surf_comb, static, interpolated_static_vars):
    i = 1  
    batch = Batch(
        surf_vars={
            # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
            # batch dimension of size one.
            "2t": torch.from_numpy(surf_comb["t2m"].values[[i - 1, i]][None]),
            "10u": torch.from_numpy(surf_comb["u10"].values[[i - 1, i]][None]),
            "10v": torch.from_numpy(surf_comb["v10"].values[[i - 1, i]][None]),
            "msl": torch.from_numpy(surf_comb["msl"].values[[i - 1, i]][None]),
            "fire": torch.from_numpy(surf_comb["fire"].values[[i - 1, i]][None]),
            "lst": torch.from_numpy(surf_comb["lst"].values[[i - 1, i]][None]),
        },
        # static_vars={
        #     # The static variables are constant, so we just get them for the first time.
        #     "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        #     "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        #     "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
        # },
        static_vars=static ,
        atmos_vars={
            "t": torch.from_numpy(atmos_comb["t"].values[[i - 1, i]][None]),
            "u": torch.from_numpy(atmos_comb["u"].values[[i - 1, i]][None]),
            "v": torch.from_numpy(atmos_comb["v"].values[[i - 1, i]][None]),
            "q": torch.from_numpy(atmos_comb["q"].values[[i - 1, i]][None]),
            "z": torch.from_numpy(atmos_comb["z"].values[[i - 1, i]][None]),
        },
        metadata=Metadata(
            lat=torch.from_numpy(surf_comb.latitude.values),
            lon=torch.from_numpy(surf_comb.longitude.values),
            # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
            # `datetime.datetime`s. Note that this needs to be a tuple of length one:
            # one value for every batch element.
            time=(surf_comb.valid_time.values.astype("datetime64[s]").tolist()[i],),
            atmos_levels=tuple(int(level) for level in atmos_comb.pressure_level.values),
        )
        # metadata=metadata
    )
    
    j = 2  # Select this time index in the downloaded data.

    surf_size = (720, 1440)
    atmos_size = (720, 1440)

    surf_vars={
        # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
        # batch dimension of size one.
        "2t": torch.from_numpy(surf_comb["t2m"].values[[j]][None]),
        "10u": torch.from_numpy(surf_comb["u10"].values[[j]][None]),
        "10v": torch.from_numpy(surf_comb["v10"].values[[j]][None]),
        "msl": torch.from_numpy(surf_comb["msl"].values[[j]][None]),
        "fire": torch.from_numpy(surf_comb["fire"].values[[j]][None]),
        # ************** Comment out LST
        "lst": torch.from_numpy(surf_comb["lst"].values[[j]][None]),
    }
    # static_vars={
    #     # The static variables are constant, so we just get them for the first time.
    #     "z": torch.from_numpy(static_vars_ds["z"].values[0]),
    #     "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
    #     "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
    # }
    # static_vars=static 
    atmos_vars={
        "t": torch.from_numpy(atmos_comb["t"].values[[j]][None]),
        "u": torch.from_numpy(atmos_comb["u"].values[[j]][None]),
        "v": torch.from_numpy(atmos_comb["v"].values[[j]][None]),
        "q": torch.from_numpy(atmos_comb["q"].values[[j]][None]),
        "z": torch.from_numpy(atmos_comb["z"].values[[j]][None]),
    }

    output_size = (720, 1440)

    interpolated_surf_vars = {
        key: F.interpolate(value, size=surf_size, mode='bilinear', align_corners=False)
        for key, value in surf_vars.items()
    }

    interpolated_atmos_vars = {
        key: F.interpolate(value.view(-1, 1, 721, 1440), size=atmos_size, mode='bilinear', align_corners=False).view(value.shape[0], value.shape[1], value.shape[2], 720, 1440)
        for key, value in atmos_vars.items()
    }

    # interpolated_static_vars = {
    #     key: F.interpolate(value.unsqueeze(0).unsqueeze(0), size=output_size, mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
    #     for key, value in static_vars.items()
    # }


    batch2 = Batch(
        surf_vars = interpolated_surf_vars,
        static_vars=interpolated_static_vars,
        atmos_vars=interpolated_atmos_vars,
        metadata=Metadata(
            lat=torch.from_numpy(surf_comb.latitude.values),
            lon=torch.from_numpy(surf_comb.longitude.values),
            # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
            # `datetime.datetime`s. Note that this needs to be a tuple of length one:
            # one value for every batch element.
            time=(surf_comb.valid_time.values.astype("datetime64[s]").tolist()[j],),
            atmos_levels=tuple(int(level) for level in atmos_comb.pressure_level.values),
        ),
        # metadata=metadata
    )
    return batch, batch2

In [9]:
class AuroraDataset(Dataset):
    def __init__(self, download_path, variables_to_keep , starty, endy):
        """
        Args:
            folder_path (str): Path to the folder containing the daily files.
            transform (callable, optional): A function/transform to apply to the data.
        """
        with xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4") as static_vars_ds:
            self.static={
                # The static variables are constant, so we just get them for the first time.
                "z": torch.from_numpy(static_vars_ds["z"].values[0]),
                "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
                "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
            }
            self.interp_static = {
                key: F.interpolate(value.unsqueeze(0).unsqueeze(0), size=(720, 1440), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                for key, value in self.static.items()
            }
        self.atmos_files = glob.glob(f"../{download_path}/atmospheric/*/*.nc") 
        self.surf_files = glob.glob(f"../{download_path}/fle/*.nc") 
        
        self.window = 3
        self.variables_to_keep = variables_to_keep
        
        self.date_range = pd.date_range(start=f'{starty}-01-01', end=f'{endy}-12-31', freq='D')
        
        # self.file_names = sorted(os.listdir(folder_path))  # Sort to ensure correct order
        # self.transform = transform

    def __len__(self):
        """Returns the total number of files (days)."""
        return len(self.surf_files) - self.window + 1

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the file to load.
        Returns:
            (Tensor, Tensor): Features and labels for the given day.
        """
#         file_path = os.path.join(self.folder_path, self.file_names[idx])
        
        #                       
        atmos, surf = data_retreive(self.atmos_files, self.surf_files, self.date_range
                             , self.window, idx, self.variables_to_keep)
        
        # metadata=Metadata(
        #     lat=torch.from_numpy(surf.latitude.values),
        #     lon=torch.from_numpy(surf.longitude.values),
        #     # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        #     # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        #     # one value for every batch element.
        #     time=(surf.valid_time.values.astype("datetime64[s]").tolist()[1],),
        #     atmos_levels=tuple(int(level) for level in atmos.pressure_level.values),
        # )
        # Assuming the last column is the label
        features, labels = feature_label_batch(atmos, surf, self.static, self.interp_static)
         # = label_batch(atmos, surf, self.interp_static, metadata) # metadata technically wrong for labels

        # Convert to tensors
        return (
            features, labels
        )


In [10]:
def aurora_collate(batch):
    """
    Custom collate function for handling Batch objects as features and labels.

    Args:
        batch (list of tuples): Each tuple contains (features, labels), where
                                features and labels are instances of `Batch`.

    Returns:
        Batched features and labels.
    """
    # Separate features and labels from the batch
    features_list, labels_list = zip(*batch)

    # Function to batch the variables in each component of the `Batch` object
    def collate_batch_components(batch_list, key):
        return {
            var_key: torch.stack([getattr(b, key)[var_key] for b in batch_list])
            for var_key in getattr(batch_list[0], key)}

    # Combine `surf_vars` and `atmos_vars` for features and labels
    batched_features = Batch(
        surf_vars=collate_batch_components(features_list, "surf_vars"),
        static_vars=collate_batch_components(features_list, "static_vars"),
        atmos_vars=collate_batch_components(features_list, "atmos_vars"),
        metadata=Metadata(
            lat=torch.stack([f.metadata.lat for f in features_list]),
            lon=torch.stack([f.metadata.lon for f in features_list]),
            time=[f.metadata.time for f in features_list],
            atmos_levels=[f.metadata.atmos_levels for f in features_list],
        ),
    )

    batched_labels = Batch(
        surf_vars=collate_batch_components(labels_list, "surf_vars"),
        static_vars=collate_batch_components(features_list, "static_vars"),
        atmos_vars=collate_batch_components(labels_list, "atmos_vars"),
        metadata=Metadata(
            lat=torch.stack([l.metadata.lat for l in labels_list]),
            lon=torch.stack([l.metadata.lon for l in labels_list]),
            time=[l.metadata.time for l in labels_list],
            atmos_levels=[l.metadata.atmos_levels for l in labels_list],
        ),
    )

    return features_list[0], labels_list[0]


## run as single block

In [2]:
# Normalisation means:
# locations["fire"] = 0.0
# locations["lst"] = 11484.10859

# Normalisation standard deviations:
# scales["fire"] = 1.0
# scales["lst"] = 8109.33224

# variables_to_keep = ['u10', 'v10', 't2m', 'msl', 'fire']
# variables_to_keep = ['u10', 'v10', 't2m', 'msl', 'lst', 'fire']

def data_retreive ( atmos_files, surf_files, date_range, days, idx , variables_to_keep = None ):
    date_window = date_range[idx:idx + days]
    # print(date_range)
    # print(idx)
    # print(days)
    atmos_regex = []
    surf_regex = []
    for day in range(days):
        # note that no need to look at download path here because its already accounted for when globing inside of aurora dataset
        # atmos_regex.append( re.compile( rf"\.\./{download_path}/atmospheric/{date_window[day].strftime('%Y%m')}/atmospheric_{date_window[day].strftime('%Y%m%d')}\.nc$"))
        # surf_regex.append( re.compile( rf"\.\./{download_path}/fle/surf_{date_window[day].strftime('%Y-%m-%d')}\.nc$"))
        # atmos_regex.append( re.compile( rf".*/{download_path}.*/{date_window[day].strftime('%Y%m')}/atmospheric_{date_window[day].strftime('%Y%m%d')}\.nc$"))
        # surf_regex.append( re.compile( rf".*/{download_path}.*/surf_{date_window[day].strftime('%Y-%m-%d')}\.nc$"))
        atmos_regex.append( re.compile( rf".*/{date_window[day].strftime('%Y%m')}/atmospheric_{date_window[day].strftime('%Y%m%d')}\.nc$"))
        surf_regex.append( re.compile( rf".*/surf_{date_window[day].strftime('%Y-%m-%d')}\.nc$"))

    atmos_filt = [f for f in atmos_files if any(d.match(f) for d in atmos_regex)]
    surf_filt = [f for f in surf_files if any(d.match(f) for d in surf_regex)]
    atmos_list = []
    surf_list = []
    # print(atmos_filt)
    # print("surf")
    # print(surf_filt)
    ds = xr.open_dataset(atmos_filt[0])
    ds.close()

    # for day in range(len(atmos_filt)):
    for day in range(days):
        # surf_cur = None  # Initialize to None
        # atmos_cur = None  # Initialize to None
        # print(atmos_filt[day])
        # with xr.open_dataset(atmos_filt[day])as atmos_cur, xr.open_dataset(surf_filt[day]) as surf_cur :
        try: 
        # with xr.open_dataset(atmos_filt[day])as atmos_cur, xr.open_dataset(surf_filt[day]) as surf_cur :
        # with xr.open_dataset(atmos_filt[day])as atmos_cur, xr.open_dataset(surf_filt[day]) as surf_cur :
            atmos_cur = xr.open_dataset(atmos_filt[day])
            surf_cur = xr.open_dataset(surf_filt[day])
        

            # code for dropping variables. makes code run slower. interpolation so fast that it doest matter. 
            if variables_to_keep is not None:
                surf_cur = surf_cur.drop_vars([var for var in surf_cur.data_vars if var not in variables_to_keep])

            atmos_list.append(atmos_cur.load())
            surf_list.append(surf_cur.load())
            
        finally: 
            if atmos_cur is not None:
                atmos_cur.close()
            if surf_cur is not None:
                surf_cur.close()

    atmos_comb = xr.concat(atmos_list, dim="valid_time")
    surf_comb = xr.concat(surf_list, dim="valid_time")
    return atmos_comb, surf_comb

def feature_label_batch(atmos_comb, surf_comb, static, interpolated_static_vars):
    i = 1  
    batch = Batch(
        surf_vars={
            # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
            # batch dimension of size one.
            "2t": torch.from_numpy(surf_comb["t2m"].values[[i - 1, i]][None]),
            "10u": torch.from_numpy(surf_comb["u10"].values[[i - 1, i]][None]),
            "10v": torch.from_numpy(surf_comb["v10"].values[[i - 1, i]][None]),
            "msl": torch.from_numpy(surf_comb["msl"].values[[i - 1, i]][None]),
            "fire": torch.from_numpy(surf_comb["fire"].values[[i - 1, i]][None]),
            "lst": torch.from_numpy(surf_comb["lst"].values[[i - 1, i]][None]),
        },
        # static_vars={
        #     # The static variables are constant, so we just get them for the first time.
        #     "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        #     "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        #     "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
        # },
        static_vars=static ,
        atmos_vars={
            "t": torch.from_numpy(atmos_comb["t"].values[[i - 1, i]][None]),
            "u": torch.from_numpy(atmos_comb["u"].values[[i - 1, i]][None]),
            "v": torch.from_numpy(atmos_comb["v"].values[[i - 1, i]][None]),
            "q": torch.from_numpy(atmos_comb["q"].values[[i - 1, i]][None]),
            "z": torch.from_numpy(atmos_comb["z"].values[[i - 1, i]][None]),
        },
        metadata=Metadata(
            lat=torch.from_numpy(surf_comb.latitude.values),
            lon=torch.from_numpy(surf_comb.longitude.values),
            # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
            # `datetime.datetime`s. Note that this needs to be a tuple of length one:
            # one value for every batch element.
            time=(surf_comb.valid_time.values.astype("datetime64[s]").tolist()[i],),
            atmos_levels=tuple(int(level) for level in atmos_comb.pressure_level.values),
        )
        # metadata=metadata
    )
    
    print(np.unique(batch.surf_vars["fire"].cpu().numpy()))
    
    j = 2  # Select this time index in the downloaded data.

    surf_size = (720, 1440)
    atmos_size = (720, 1440)

    surf_vars={
        # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
        # batch dimension of size one.
        "2t": torch.from_numpy(surf_comb["t2m"].values[[j]][None]),
        "10u": torch.from_numpy(surf_comb["u10"].values[[j]][None]),
        "10v": torch.from_numpy(surf_comb["v10"].values[[j]][None]),
        "msl": torch.from_numpy(surf_comb["msl"].values[[j]][None]),
        "fire": torch.from_numpy(surf_comb["fire"].values[[j]][None]),
        # ************** Comment out LST
        "lst": torch.from_numpy(surf_comb["lst"].values[[j]][None]),
    }
    print(np.unique(surf_vars["fire"].cpu().numpy()))
    atmos_vars={
        "t": torch.from_numpy(atmos_comb["t"].values[[j]][None]),
        "u": torch.from_numpy(atmos_comb["u"].values[[j]][None]),
        "v": torch.from_numpy(atmos_comb["v"].values[[j]][None]),
        "q": torch.from_numpy(atmos_comb["q"].values[[j]][None]),
        "z": torch.from_numpy(atmos_comb["z"].values[[j]][None]),
    }

    output_size = (720, 1440)


    # interpolated_surf_vars = {
    #     key: (F.interpolate(value, size=surf_size, mode='bilinear', align_corners=False) >= 0.5).int()
    #     if key == "fire" else  F.interpolate(value, size=surf_size, mode='bilinear', align_corners=False)
    #     for key, value in surf_vars.items()
    # }

    interpolated_surf_vars = {
        key: F.interpolate(value, size=surf_size, mode='bilinear', align_corners=False)
        for key, value in surf_vars.items()
    }

    print("before count", surf_vars["fire"].count_nonzero(y == 1) )
    print("after count", interpolated_surf_vars["fire"].count_nonzero(y == 1) )
    
    interpolated_atmos_vars = {
        key: F.interpolate(value.view(-1, 1, 721, 1440), size=atmos_size, mode='bilinear', align_corners=False).view(value.shape[0], value.shape[1], value.shape[2], 720, 1440)
        for key, value in atmos_vars.items()
    }

    batch2 = Batch(
        surf_vars = interpolated_surf_vars,
        static_vars=interpolated_static_vars,
        atmos_vars=interpolated_atmos_vars,
        metadata=Metadata(
            lat=torch.from_numpy(surf_comb.latitude.values),
            lon=torch.from_numpy(surf_comb.longitude.values),
            # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
            # `datetime.datetime`s. Note that this needs to be a tuple of length one:
            # one value for every batch element.
            time=(surf_comb.valid_time.values.astype("datetime64[s]").tolist()[j],),
            atmos_levels=tuple(int(level) for level in atmos_comb.pressure_level.values),
        ),
        # metadata=metadata
    )
    return batch, batch2

class AuroraDataset(Dataset):
    def __init__(self, download_path, variables_to_keep , date_range):
        """
        Args:
            folder_path (str): Path to the folder containing the daily files.
            transform (callable, optional): A function/transform to apply to the data.
        """
        self.sumides = 5
        with xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4") as static_vars_ds :
            # static_vars_ds = pstatic_vars_ds
            
            prestatic={
                # The static variables are constant, so we just get them for the first time.
                "z": torch.from_numpy(np.copy(static_vars_ds["z"].values[0])),
                "slt": torch.from_numpy(np.copy(static_vars_ds["slt"].values[0])),
                "lsm": torch.from_numpy(np.copy(static_vars_ds["lsm"].values[0])),
            }
            # prestatic={
            #     # The static variables are constant, so we just get them for the first time.
            #     "z": torch.from_numpy(static_vars_ds["z"].values[0].copy()),
            #     "slt": torch.from_numpy(static_vars_ds["slt"].values[0].copy()),
            #     "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0].copy()),
            # }
            print(prestatic)
            self.static= prestatic
            self.interp_static = {
                key: F.interpolate(value.unsqueeze(0).unsqueeze(0), size=(720, 1440), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                for key, value in prestatic.items()
            }

        # static_vars_ds = xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4")
        # static={
        #         # The static variables are constant, so we just get them for the first time.
        #         "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        #         "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        #         "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
        #     }
        # self.interp_static = {
        #         key: F.interpolate(value.unsqueeze(0).unsqueeze(0), size=(720, 1440), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
        #         for key, value in static.items()
        #     }
        # static_vars_ds.close()

        
        # with xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4") as static_vars_ds:
        #     self.static={
        #         # The static variables are constant, so we just get them for the first time.
        #         "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        #         "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        #         "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
        #     }
        #     self.interp_static = {
        #         key: F.interpolate(value.unsqueeze(0).unsqueeze(0), size=(720, 1440), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
        #         for key, value in self.static.items()
        #     }
        # self.atmos_files = glob.glob(f"../{download_path}/atmospheric/*/*.nc") 
        # self.surf_files = glob.glob(f"../{download_path}/fle/*.nc") 
        
        self.window = 3
        self.variables_to_keep = variables_to_keep
        
        self.date_range = date_range
        self.download_path = download_path
        
        # self.file_names = sorted(os.listdir(folder_path))  # Sort to ensure correct order
        # self.transform = transform

    def __len__(self):
        """Returns the total number of files (days)."""
        # return len(self.surf_files) - self.window + 1
        return len(date_range) - self.window + 1

    def __getitem__(self, idx):
        """
        Args:
            idx (int): Index of the file to load.
        Returns:
            (Tensor, Tensor): Features and labels for the given day.
        """
#         file_path = os.path.join(self.folder_path, self.file_names[idx])
        
        #                       
        atmos, surf = data_retreive(self.atmos_files, self.surf_files, self.date_range
                             , self.window, idx,  self.variables_to_keep)
        
        # metadata=Metadata(
        #     lat=torch.from_numpy(surf.latitude.values),
        #     lon=torch.from_numpy(surf.longitude.values),
        #     # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        #     # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        #     # one value for every batch element.
        #     time=(surf.valid_time.values.astype("datetime64[s]").tolist()[1],),
        #     atmos_levels=tuple(int(level) for level in atmos.pressure_level.values),
        # )
        # Assuming the last column is the label
        features, labels = feature_label_batch(atmos, surf, self.static, self.interp_static)
         # = label_batch(atmos, surf, self.interp_static, metadata) # metadata technically wrong for labels
        
        print("in getitem feature:", np.unique(features.surf_vars["fire"].cpu().numpy()))
        print("in getitem label:", np.unique(labels.surf_vars["fire"].cpu().numpy()))
        # Convert to tensors
        return (
            features, labels
        )

def aurora_collate(batch):
    """
    Custom collate function for handling Batch objects as features and labels.

    Args:
        batch (list of tuples): Each tuple contains (features, labels), where
                                features and labels are instances of `Batch`.

    Returns:
        Batched features and labels.
    """
    # Separate features and labels from the batch
    features_list, labels_list = zip(*batch)

    # Function to batch the variables in each component of the `Batch` object
    def collate_batch_components(batch_list, key):
        return {
            var_key: torch.stack([getattr(b, key)[var_key] for b in batch_list])
            for var_key in getattr(batch_list[0], key)}

    # Combine `surf_vars` and `atmos_vars` for features and labels
    batched_features = Batch(
        surf_vars=collate_batch_components(features_list, "surf_vars"),
        static_vars=collate_batch_components(features_list, "static_vars"),
        atmos_vars=collate_batch_components(features_list, "atmos_vars"),
        metadata=Metadata(
            lat=torch.stack([f.metadata.lat for f in features_list]),
            lon=torch.stack([f.metadata.lon for f in features_list]),
            time=[f.metadata.time for f in features_list],
            atmos_levels=[f.metadata.atmos_levels for f in features_list],
        ),
    )

    batched_labels = Batch(
        surf_vars=collate_batch_components(labels_list, "surf_vars"),
        static_vars=collate_batch_components(features_list, "static_vars"),
        atmos_vars=collate_batch_components(labels_list, "atmos_vars"),
        metadata=Metadata(
            lat=torch.stack([l.metadata.lat for l in labels_list]),
            lon=torch.stack([l.metadata.lon for l in labels_list]),
            time=[l.metadata.time for l in labels_list],
            atmos_levels=[l.metadata.atmos_levels for l in labels_list],
        ),
    )

    return features_list[0], labels_list[0]


## dataset/loader testing

In [5]:
starty = 2015     ## start of 2015 to end of 2021
endy = 2021           
date_range = pd.date_range(start=f'{starty}-01-01', end=f'{endy}-12-31', freq='D')
variables_to_keep = ['u10', 'v10', 't2m', 'msl', 'lst', 'fire']

download_path = "datasets/aurora"


dataset = AuroraDataset(download_path, variables_to_keep , starty, endy)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=aurora_collate)

TypeError: AuroraDataset.__init__() takes 4 positional arguments but 5 were given

In [ ]:
for features, labels in dataloader:
    print(features.surf_vars["2t"].shape)  # Ensure shapes are correct
    print(labels.surf_vars["2t"].shape)
    break

In [ ]:
for batch_data, batch_labels in dataloader:
    print(batch_labels)

In [ ]:
# testing cell

print(batch.surf_vars['2t'].size())
print(interpolated_surf_vars['2t'].size())
print(interpolated_atmos_vars['t'].size())

# print(.size())
# print(f" size: {.size()}")
print(atmos_vars['t'].view(-1, 1, 721, 1440).size())
print(atmos_vars['t'].view(-1, 1, 721, 1440).shape[2])

# interpolated_surf_vars
# surf_vars

# print(atmos_vars['t'].size())
# print(atmos_vars['u'].size())
# print(atmos_vars['v'].size())
# print(atmos_vars['q'].size())
# print(atmos_vars['z'].size())
# torch.from_numpy(surf_comb["t2m"].values[[i - 1, i]][None])

print(static_vars['z'].unsqueeze(0).unsqueeze(0).squeeze(0).size())

# atmos_vars.items()
print(batch2.surf_vars['2t'].size())
print(interpolated_surf_vars['2t'].size())
batch2.surf_vars['2t']

print(f"batch2 size: {batch2.surf_vars['2t'].size()}")
print(f"pred size: {pred.surf_vars['2t'].size()}")
batch2.surf_vars['2t']

In [ ]:
# interpolated_surf_vars['2t']
# pred.surf_vars['2t']
batch.surf_vars['2t']

In [ ]:
test =  torch.from_numpy(surf_vars_ds["lst"].values[[i - 1, i]][None])
test.mean()
test.std()

## model 

In [3]:
device = "cuda"

def loss_func(batch, pred):
    crit = nn.L1Loss()
    batch = batch.to('cuda')
    pred = pred.to('cuda')
    v_w = {}
    alpha = 0.25
    beta = 1
    dataset_weight = 2
    total_vars = len(batch.surf_vars) + len(batch.atmos_vars)
    v_w["2t"] = 3.0
    v_w["10u"] = 0.77
    v_w["10v"] = 0.66
    v_w["msl"] = 1.5
    v_w["z"] = 2.8
    v_w["q"] = 0.78
    v_w["t"] = 1.7
    v_w["u"] = 0.87
    v_w["v"] = 0.6
    v_w["fire"] = 1
    # ******* Comment out LST for now 
    # no need to comment them out as below code only checks for existing ones
    v_w["lst"] = 1
    l_s = torch.zeros(1, dtype=torch.float32, device=device)
    l_a = torch.zeros(1, dtype=torch.float32, device=device)
    for key in batch.surf_vars.keys():
        l_s += v_w[key]*crit(pred.surf_vars[key] + 1e-8, batch.surf_vars[key] + 1e-8)/(721*1440)
    for key in batch.atmos_vars.keys():
        l_a += v_w[key]*crit(pred.atmos_vars[key] + 1e-8, batch.atmos_vars[key] + 1e-8)/(721*1440*13)
    
    l = (2/total_vars)*(l_s * alpha) + (l_a*beta)

    return l   

In [9]:
# del locations["fire"]
# del scales["fire"]
# locations
# print(locations)

when changing things, change out instances of it in 5, location/scales, model itself.

In [4]:
# Normalisation means:
locations["fire"] =1e-8
locations["lst"] = 11484.10859

# Normalisation standard deviations:
scales["fire"] = 1.0
scales["lst"] = 8109.33224
starty = 2015     ## start of 2015 to end of 2021
endy = 2015           
date_range = pd.date_range(start=f'{starty}-01-01', end=f'{endy}-01-05', freq='D')
date_range = pd.date_range(start=f'{starty}-01-01', end=f'{endy}-12-31', freq='D')
# if keyerror, dleete variable
# variables_to_keep = ['u10', 'v10', 't2m', 'msl', 'fire']
# no need to worry about dong less. manually managug from 5 is way to edit for now
variables_to_keep = ['u10', 'v10', 't2m', 'msl', 'lst', 'fire']

download_path = "globfire-gooddata/data"
# download_path = "globfire-gooddata"

dataset = AuroraDataset(download_path, variables_to_keep , date_range)

train_size = int(0.8 * len(dataset))  
train_size = int(0.95 * len(dataset))  
test_size = len(dataset) - train_size 
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=aurora_collate)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=aurora_collate)

{'z': tensor([[ 3.2275e-01,  3.2275e-01,  3.2275e-01,  ...,  3.2275e-01,
          3.2275e-01,  3.2275e-01],
        [ 4.6157e+00,  4.6040e+00,  4.5884e+00,  ...,  4.7915e+00,
          4.7329e+00,  4.6743e+00],
        [-1.7710e+00, -1.7163e+00, -1.7085e+00,  ..., -1.5093e+00,
         -1.6226e+00, -1.7124e+00],
        ...,
        [ 2.6958e+04,  2.6961e+04,  2.6965e+04,  ...,  2.6950e+04,
          2.6953e+04,  2.6956e+04],
        [ 2.7163e+04,  2.7165e+04,  2.7167e+04,  ...,  2.7159e+04,
          2.7160e+04,  2.7162e+04],
        [ 2.7355e+04,  2.7355e+04,  2.7355e+04,  ...,  2.7355e+04,
          2.7355e+04,  2.7355e+04]]), 'slt': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'lsm': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0.,

In [14]:
# xr.set_options(file_cache_maxsize=128)

# for batch in dataloader:
#     print(1)

In [17]:
# import netCDF4 as nc
# try:
#     ds = nc.Dataset('/home/ubuntu/globfire-gooddata/data/atmospheric/201503/atmospheric_20150302.nc')
#     print(ds)
# except Exception as e:
#     print(f"Error: {e}")

asdf = xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4")
asdf2={
                # The static variables are constant, so we just get them for the first time.
                "z": torch.from_numpy(asdf["z"].values[0]),
                "slt": torch.from_numpy(asdf["slt"].values[0]),
                "lsm": torch.from_numpy(asdf["lsm"].values[0]),
            }

print(asdf2)
asdf3 = asdf2
buh = {
                key: F.interpolate(value.unsqueeze(0).unsqueeze(0), size=(720, 1440), mode='bilinear', align_corners=False).squeeze(0).squeeze(0)
                for key, value in asdf2.items()
            }
asdf.close()



{'z': tensor([[ 3.2275e-01,  3.2275e-01,  3.2275e-01,  ...,  3.2275e-01,
          3.2275e-01,  3.2275e-01],
        [ 4.6157e+00,  4.6040e+00,  4.5884e+00,  ...,  4.7915e+00,
          4.7329e+00,  4.6743e+00],
        [-1.7710e+00, -1.7163e+00, -1.7085e+00,  ..., -1.5093e+00,
         -1.6226e+00, -1.7124e+00],
        ...,
        [ 2.6958e+04,  2.6961e+04,  2.6965e+04,  ...,  2.6950e+04,
          2.6953e+04,  2.6956e+04],
        [ 2.7163e+04,  2.7165e+04,  2.7167e+04,  ...,  2.7159e+04,
          2.7160e+04,  2.7162e+04],
        [ 2.7355e+04,  2.7355e+04,  2.7355e+04,  ...,  2.7355e+04,
          2.7355e+04,  2.7355e+04]]), 'slt': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]]), 'lsm': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0.,

In [ ]:
# from aurora import Aurora, rollout
# from aurora.normalisation import locations, scales

model = Aurora(
    use_lora=False,
    autocast=True,      # reduces memory usage
    surf_vars=("2t", "10u", "10v", "msl", "fire","lst"),
    static_vars=("lsm", "z", "slt"),
    atmos_vars=("z", "u", "v", "t", "q"),
)
# model = Aurora(
#     use_lora=False,
#     autocast=True,      # reduces memory usage
#     # surf_vars=("2t", "10u", "10v", "msl"),
#     surf_vars=("2t", "10u", "10v", "msl", "fire"),
#     static_vars=("lsm", "z", "slt"),
#     atmos_vars=("z", "u", "v", "t", "q"),
# )

model.load_checkpoint("microsoft/aurora", "aurora-0.25-pretrained.ckpt", strict=False)
model = model.to("cuda")


In [ ]:
# training 
model.train()
model.configure_activation_checkpointing()
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=5e-6)
# scaler = torch.cuda.amp.GradScaler() #
scaler = torch.amp.GradScaler('cuda') #

ctr = 1
num_epochs = 1

# torch.cuda.empty_cache()
for epoch in range(num_epochs):  # Loop over epochs
    # model.train()  # Set model to training mode
    for batch in dataloader:  # Iterate over batches
        # Unpack the batch
        features, labels = batch  # Ensure your Dataset and collate function return the right format
        # Move data to the appropriate device
        # features = {key: val.to(device) for key, val in features.items()}  # For dict-based features
        labels = labels.to(device)
        features = features.to(device)
        # print(features.surf_vars["lst"])
        # print(features)
        
        optimizer.zero_grad()  # Zero the gradients

        with torch.cuda.amp.autocast():
            # Forward pass
            # outputs = model(features)
            pred = model.forward(features)
            # print(pred.surf_vars["lst"])
            # pred is already on gpu 
            # pred = pred.to(device)[
            # print(features)
            # Compute the loss
            loss = loss_func(labels,pred)
        
        # Backward pass and optimization
        scaler.scale(loss).backward()  # Backpropagate  nan occurs here 
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! gradient clipping!
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # optimizer.step()  # Update the weights
        scaler.step(optimizer)
        scaler.update()
        # torch.cuda.empty_cache()
        print(f"computed batch {ctr}")
        ctr += 1
        
        # Debug memory usage
        print(f"Allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
        print(f"Reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
    break
    

In [19]:
## freeing memory after restarting model

with torch.no_grad():
    torch.cuda.empty_cache()

import gc
torch.cuda.empty_cache()
gc.collect()

287

### some debug stuff (not important for runing)

In [ ]:
# model training 

model.train()
model.configure_activation_checkpointing()
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=5e-6)

model = model.to(device)
model.train()
model.configure_activation_checkpointing()
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

num_epochs = 1
torch.cuda.empty_cache()
for epoch in range(num_epochs):  # Loop over epochs
    # model.train()  # Set model to training mode
    for batch in dataloader:  # Iterate over batches
        # Unpack the batch
        features, labels = batch  # Ensure your Dataset and collate function return the right format
        # Move data to the appropriate device
        # features = {key: val.to(device) for key, val in features.items()}  # For dict-based features
        labels = labels.to(device)
        # features = features.to(device)

        # Forward pass
        # outputs = model(features)
        pred = model.forward(features)
        # pred is already on gpu 
        # pred = pred.to(device)[
        print(pred)
        # Compute the loss
        loss = loss_func(labels,pred)
        
        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagate
        optimizer.step()  # Update the weights
        torch.cuda.empty_cache()
        print("computed first batch")

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
    break

In [15]:
# model training 

import torch
import torch.nn as nn
import torch.optim as optim

model.train()
model.configure_activation_checkpointing()
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=5e-6)

model = model.to(device)
model.train()
model.configure_activation_checkpointing()
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

num_epochs = 1
torch.cuda.empty_cache()
for epoch in range(num_epochs):  # Loop over epochs
    # model.train()  # Set model to training mode
    for batch in dataloader:  # Iterate over batches
        # Unpack the batch
        features, labels = batch  # Ensure your Dataset and collate function return the right format
        # Move data to the appropriate device
        # features = {key: val.to(device) for key, val in features.items()}  # For dict-based features
        labels = labels.to(device)
        # features = features.to(device)

        # Forward pass
        # outputs = model(features)
        pred = model.forward(features)
        # pred is already on gpu 
        # pred = pred.to(device)
        print(pred)
        # Compute the loss
        loss = loss_func(labels,pred)
        
        # Backward pass and optimization
        optimizer.zero_grad()  # Zero the gradients
        loss.backward()  # Backpropagate
        optimizer.step()  # Update the weights
        torch.cuda.empty_cache()
        print("computed first batch")

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
    break

Batch(surf_vars={'2t': tensor([[[[248.2111, 250.3261, 250.2283,  ..., 249.8091, 250.0388, 247.3506],
          [256.5222, 255.5726, 255.3369,  ..., 255.0719, 255.2536, 245.8392],
          [252.1862, 253.3263, 249.0282,  ..., 253.4009, 249.3633, 248.7849],
          ...,
          [243.7909, 243.7615, 243.8710,  ..., 243.7093, 243.9118, 244.1356],
          [244.0923, 243.9736, 244.1088,  ..., 243.8568, 243.9820, 244.0134],
          [243.9003, 243.9125, 243.7339,  ..., 243.9700, 243.8595, 243.8047]]]],
       device='cuda:0', grad_fn=<AddBackward0>), '10u': tensor([[[[  3.5809,   3.3212,   1.3584,  ...,   3.0293,   1.0162,   3.7811],
          [ -7.9823,  -7.5491,  -9.8704,  ...,  -7.3736,  -9.8940,  -7.1199],
          [ -4.5390,  -4.3756, -10.2827,  ...,  -4.1192, -10.1291,  -8.2069],
          ...,
          [ -3.3834,  -3.3105,  -3.2968,  ...,  -3.2817,  -3.2331,  -3.3291],
          [ -3.0977,  -3.0532,  -3.0555,  ...,  -3.1011,  -3.0014,  -3.0439],
          [ -2.4023,  -2.4306,

/home/ubuntu/.local/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


computed first batch
Batch(surf_vars={'2t': tensor([[[[246.5566, 249.0670, 248.9123,  ..., 248.7042, 248.8638, 246.1772],
          [254.5370, 254.2614, 253.9191,  ..., 253.9032, 253.9358, 248.8828],
          [250.2312, 251.7356, 247.2331,  ..., 251.6860, 247.5735, 246.9567],
          ...,
          [245.6753, 245.1755, 245.6828,  ..., 246.0768, 246.3116, 246.4983],
          [247.3932, 246.8410, 247.1264,  ..., 246.4315, 246.4003, 246.4061],
          [245.2274, 245.0392, 245.4596,  ..., 246.7330, 246.5004, 246.3099]]]],
       device='cuda:0', grad_fn=<AddBackward0>), '10u': tensor([[[[  2.6317,   2.3470,   0.6036,  ...,   1.8345,   0.3006,   2.6186],
          [ -8.5061,  -7.9438, -10.3034,  ...,  -7.3911,  -9.7658,  -7.0160],
          [ -5.3238,  -5.0784, -10.9660,  ...,  -4.7888, -10.6551,  -8.6438],
          ...,
          [ -4.4275,  -4.4794,  -4.4256,  ...,  -3.8151,  -3.9774,  -4.1757],
          [ -3.8242,  -3.6514,  -3.3166,  ...,  -4.1819,  -4.0720,  -4.0007],
         

/home/ubuntu/.local/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: cuDNN SDPA backward got grad_output.strides() != output.strides(), attempting to materialize a grad_output with matching strides... (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/MHA.cpp:674.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


computed first batch
Epoch [1/1], Loss: 0.0017


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
# !export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:64 3.17 112

In [ ]:
print(os.getenv("PYTORCH_CUDA_ALLOC_CONF"))

In [ ]:
print(torch.cuda.memory_summary())

### predicting on new data

In [27]:
model = Aurora(
    use_lora=False,
    autocast=True,      # reduces memory usage
    surf_vars=("2t", "10u", "10v", "msl", "fire","lst"),
    static_vars=("lsm", "z", "slt"),
    atmos_vars=("z", "u", "v", "t", "q"),
)
# model = Aurora(
#     use_lora=False,
#     autocast=True,      # reduces memory usage
#     # surf_vars=("2t", "10u", "10v", "msl"),
#     surf_vars=("2t", "10u", "10v", "msl", "fire"),
#     static_vars=("lsm", "z", "slt"),
#     atmos_vars=("z", "u", "v", "t", "q"),
# )

# model.load_checkpoint("microsoft/aurora", "aurora-0.25-pretrained.ckpt", strict=False)
model.load_state_dict(torch.load("aurora_fire_weights_65.pth"), strict=False)
model = model.to("cuda")

/tmp/ipykernel_2476/1557875420.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("aurora_fire_weights_65.pth"), strict=False)


In [43]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score

accuracy = 0
precision = 0
recall = 0
f1 = 0
iou = 0
total_samples = 0

threshold = 0.5 

model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computations
    # for batch in dataloader_test:
    for batch in dataloader:
        
        features, labels = batch  # Ensure your Dataset and collate function return the right format
        

        print("label unique:", np.unique(labels.surf_vars["fire"].cpu().numpy())[:10])
        print("feature unique:",np.unique(features.surf_vars["fire"].cpu().numpy()))

        labels = labels.to(device)
        features = features.to(device)
        
        pred = model.forward(features)
        # pred = pred.to(device)
        pred_fire = pred.surf_vars["fire"].cpu().numpy().flatten()
        pred_fire = np.nan_to_num(pred_fire, nan=0)
        lab_fire = labels.surf_vars["fire"].cpu().numpy().flatten()
        # lab_fire = (lab_fire > threshold).astype(int)
        
        print("lab_fire:", lab_fire.shape, lab_fire.dtype, lab_fire)
        print("pred_fire:", pred_fire.shape, pred_fire.dtype, pred_fire)
        print(len(np.unique(lab_fire)))
        print((np.unique(lab_fire)))
        print(len(np.unique(pred_fire)))

        # Assume `preds` and `labels` are your predicted and true grids (flattened for metrics)
        # pred_fire = pred_fire.cpu().numpy().flatten()
        # lab_fire = lab_fire.cpu().numpy().flatten()
        # pred_fire = pred_fire.numpy().flatten()
        # lab_fire = lab_fire.numpy().flatten()

        # Compute metrics
        accuracy += accuracy_score(lab_fire, pred_fire)
        precision += precision_score(lab_fire, pred_fire, zero_division=1)
        recall += recall_score(lab_fire, pred_fire, zero_division=1)
        f1 += f1_score(lab_fire, pred_fire, zero_division=1)
        iou += jaccard_score(lab_fire, pred_fire)
        # total_samples += labels.size(0)
        total_samples += 1

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"IoU: {iou:.4f}")

[0. 1.]
[0. 1.]
in getitem feature: [0. 1.]
in getitem label: [0.         0.12847137 0.12985992 0.1340332  0.13542175 0.1368103
 0.14375305 0.1451416  0.14653015 0.1479187  0.14930725 0.1506958
 0.15208435 0.1534729  0.15486145 0.15625    0.15763855 0.1590271
 0.16041565 0.16320038 0.16875458 0.17014313 0.17153168 0.17292023
 0.17430878 0.17708588 0.17848206 0.18125916 0.1826477  0.18403625
 0.1854248  0.18681335 0.1882019  0.18959045 0.190979   0.19236755
 0.1937561  0.19514465 0.1965332  0.19792175 0.1993103  0.20069885
 0.2020874  0.20347595 0.2048645  0.20625305 0.2076416  0.20903015
 0.2104187  0.21180725 0.2131958  0.21458435 0.2159729  0.21736145
 0.21875    0.22013855 0.2215271  0.22291565 0.2243042  0.22569275
 0.2270813  0.22846985 0.2298584  0.23124695 0.2326355  0.23402405
 0.2354126  0.23680115 0.2395935  0.2451477  0.24653625 0.2618103
 0.2673645  0.27430725 0.2756958  0.2784729  0.27986145 0.28125
 0.2895813  0.29096985 0.2923584  0.29374695 0.2951355  0.2965393
 0.29792

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacity of 94.50 GiB of which 12.19 MiB is free. Including non-PyTorch memory, this process has 94.48 GiB memory in use. Of the allocated memory 93.64 GiB is allocated by PyTorch, and 73.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [5]:
torch.save(model.state_dict(), "aurora_fire_weights_65.pth")

### see aws id for missing line
# bucket_name = 'globfire-gooddata'
# s3.upload_file("aurora_fire_weights.pth", bucket_name, "aurora_fire_weights.pth")

ERROR! Session/line number was not unique in database. History logging moved to new session 2


NameError: name 'model' is not defined

In [ ]:
from aurora import Aurora

model = Aurora(
    use_lora=False,  # Model was not fine-tuned.
    autocast=True,  # Use AMP.
)
model.load_checkpoint("microsoft/aurora", "aurora-0.25-pretrained.ckpt")

batch = ...  # Load some data.

model = model.cuda()
model.train()
model.configure_activation_checkpointing()

pred = model.forward(batch)
loss = ...
loss.backward()

## trying to see if thing works with generic aah data 721 - 720 check


In [ ]:

static_vars_ds = xr.open_dataset(f"../{download_path}/static/static.nc", engine="netcdf4")
surf_vars_ds = xr.open_dataset(f"../{download_path}/fle/surf_2015-01-04.nc", engine="netcdf4")
atmos_vars_ds = xr.open_dataset(f"../{download_path}/atmospheric/201501/atmospheric_20150104.nc", engine="netcdf4")


i = 0  # Select this time index in the downloaded data.

batch3 = Batch(
    surf_vars={
        # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
        # batch dimension of size one.
        "2t": torch.from_numpy(surf_vars_ds["t2m"].values[[i - 1, i]][None]),
        "10u": torch.from_numpy(surf_vars_ds["u10"].values[[i - 1, i]][None]),
        "10v": torch.from_numpy(surf_vars_ds["v10"].values[[i - 1, i]][None]),
        "msl": torch.from_numpy(surf_vars_ds["msl"].values[[i - 1, i]][None])
    },
    static_vars={
        # The static variables are constant, so we just get them for the first time.
        "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
    },
    atmos_vars={
        "t": torch.from_numpy(atmos_vars_ds["t"].values[[i - 1, i]][None]),
        "u": torch.from_numpy(atmos_vars_ds["u"].values[[i - 1, i]][None]),
        "v": torch.from_numpy(atmos_vars_ds["v"].values[[i - 1, i]][None]),
        "q": torch.from_numpy(atmos_vars_ds["q"].values[[i - 1, i]][None]),
        "z": torch.from_numpy(atmos_vars_ds["z"].values[[i - 1, i]][None])
    },
    metadata=Metadata(
        lat=torch.from_numpy(surf_vars_ds.latitude.values),
        lon=torch.from_numpy(surf_vars_ds.longitude.values),
        # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        # one value for every batch element.
        time=(surf_vars_ds.valid_time.values.astype("datetime64[s]").tolist()[i],),
        atmos_levels=tuple(int(level) for level in atmos_vars_ds.pressure_level.values),
    ),
)

In [ ]:
i = 0  # Select this time index in the downloaded data.

batch4 = Batch(
    surf_vars={
        # First select time points `i` and `i - 1`. Afterwards, `[None]` inserts a
        # batch dimension of size one.
        "2t": torch.from_numpy(surf_comb["t2m"].values[[i - 1, i]][None]),
        "10u": torch.from_numpy(surf_comb["u10"].values[[i - 1, i]][None]),
        "10v": torch.from_numpy(surf_comb["v10"].values[[i - 1, i]][None]),
        "msl": torch.from_numpy(surf_comb["msl"].values[[i - 1, i]][None]),
    },
    static_vars={
        # The static variables are constant, so we just get them for the first time.
        "z": torch.from_numpy(static_vars_ds["z"].values[0]),
        "slt": torch.from_numpy(static_vars_ds["slt"].values[0]),
        "lsm": torch.from_numpy(static_vars_ds["lsm"].values[0]),
    },
    atmos_vars={
        "t": torch.from_numpy(atmos_comb["t"].values[[i - 1, i]][None]),
        "u": torch.from_numpy(atmos_comb["u"].values[[i - 1, i]][None]),
        "v": torch.from_numpy(atmos_comb["v"].values[[i - 1, i]][None]),
        "q": torch.from_numpy(atmos_comb["q"].values[[i - 1, i]][None]),
        "z": torch.from_numpy(atmos_comb["z"].values[[i - 1, i]][None]),
    },
    metadata=Metadata(
        lat=torch.from_numpy(surf_comb.latitude.values),
        lon=torch.from_numpy(surf_comb.longitude.values),
        # Converting to `datetime64[s]` ensures that the output of `tolist()` gives
        # `datetime.datetime`s. Note that this needs to be a tuple of length one:
        # one value for every batch element.
        time=(surf_comb.valid_time.values.astype("datetime64[s]").tolist()[i],),
        atmos_levels=tuple(int(level) for level in atmos_comb.pressure_level.values),
    ),
)

In [ ]:
from aurora import Aurora, rollout

model2 = Aurora(use_lora=False)  # The pretrained version does not use LoRA.
model2.load_checkpoint("microsoft/aurora", "aurora-0.25-pretrained.ckpt")

model2.eval()
model2 = model2.to("cuda")


In [ ]:

with torch.inference_mode():
    preds = [pred.to("cpu") for pred in rollout(model2, batch4, steps=2)]

model2 = model2.to("cpu")

# torch.stack(list(pred.surf_vars.values())).size()

In [ ]:
preds[1].surf_vars['2t'].size()